In [1]:
from functools import lru_cache
import numpy as np

In [2]:
def binomial_product(binomial_coeffs):
    # Initialize the product to the first binomial
    result = np.array([1, binomial_coeffs[0]])

    # Iterate through the rest of the binomials
    for a in binomial_coeffs[1:]:
        # Multiply current result by the new binomial (x + a)
        result = np.convolve(result, [1, a])

    return result

In [3]:
@lru_cache(None)
def dp_binomial_product(i, j, binomial_coeffs):
    if i == j:
        return np.array([1, binomial_coeffs[i]])

    mid = (i + j) // 2
    left_product = dp_binomial_product(i, mid, binomial_coeffs)
    right_product = dp_binomial_product(mid + 1, j, binomial_coeffs)

    return np.convolve(left_product, right_product)


def binomial_product_dp(binomial_coeffs):
    return dp_binomial_product(0, len(binomial_coeffs) - 1, tuple(binomial_coeffs))

In [4]:
def polynomial_evaluate(polynomial_coeffs: np.ndarray, x: float) -> float:
    """
    args:
        polynomial_coeffs: np.ndarray of shape (n+1,) where n is the degree of the polynomial
        x: float, the value at which to evaluate the polynomial
    returns:
        float, the value of the polynomial at x
    """
    return np.sum(
        polynomial_coeffs * np.power(x, np.arange(len(polynomial_coeffs) - 1, -1, -1))
    )


def polynomial_derivative(polynomial_coeffs):
    """
    args:
        polynomial_coeffs: np.ndarray of shape (n+1,) where n is the degree of the polynomial
    returns:
        np.ndarray of shape (n,) representing the derivative of the polynomial
    """
    if len(polynomial_coeffs) == 1:
        return np.array(
            [0], dtype=polynomial_coeffs.dtype
        )  # The derivative of a constant is 0.
    return polynomial_coeffs[:-1] * np.arange(len(polynomial_coeffs) - 1, 0, -1)

In [5]:
binomial_product([-1, -2, 10])

array([  1,   7, -28,  20])

In [6]:
binomial_product_dp([-1, -2, 10])

array([  1,   7, -28,  20])

In [7]:
coeffs = np.arange(100)
assert np.all(binomial_product(coeffs) == binomial_product_dp(coeffs))

In [8]:
stencil_idxs = np.array([-2, -1, 0, 1, 2]) * 2
x = -1

interface_idxs = np.append(stencil_idxs[0] - 1, stencil_idxs + 1)

# l1
i = 1
polynomial_coeffs = binomial_product(-np.delete(interface_idxs, i))
l1_numerator = polynomial_coeffs
l1_denominator = polynomial_evaluate(polynomial_coeffs, interface_idxs[i])
l1prime_numerator = polynomial_evaluate(polynomial_derivative(polynomial_coeffs), x)

print(l1_numerator, l1_denominator)

[  1  -3 -26  78  25 -75] 768


In [9]:
lprime_numerators = np.zeros(len(interface_idxs), dtype=type(x))
lprime_denominators = np.ones(len(interface_idxs), dtype=type(x))

for i in range(1, len(interface_idxs)):
    polynomial_coeffs = binomial_product(-np.delete(interface_idxs, i))
    li_numerator = polynomial_coeffs
    li_denominator = polynomial_evaluate(polynomial_coeffs, interface_idxs[i])
    liprime_numerator = polynomial_evaluate(polynomial_derivative(polynomial_coeffs), x)
    lprime_numerators[i] = liprime_numerator
    lprime_denominators[i] = li_denominator

print(lprime_numerators, lprime_denominators)

[   0 -192   64  192   96   64] [   1  768 -384  384 -768 3840]


In [10]:
def simplify_and_gcd(numerators: np.ndarray, denominators: np.ndarray) -> np.ndarray:
    # simplify fractions
    gcd = np.gcd(lprime_numerators, lprime_denominators)
    out_numerators = numerators // gcd
    out_denominators = denominators // gcd
    # find common denominator
    common_denominator = np.lcm.reduce(out_denominators)
    out_numerators = out_numerators * common_denominator // out_denominators
    out_denominators = out_denominators * common_denominator // out_denominators
    return out_numerators, out_denominators

In [11]:
stencil = {}
if isinstance(x, int):
    lprime_numerators, lprime_denominators = simplify_and_gcd(
        lprime_numerators, lprime_denominators
    )
    for i, idx in enumerate(stencil_idxs, start=1):
        stencil[idx] = np.sum(lprime_numerators[i:])
elif isinstance(x, float):
    fractions = lprime_numerators / lprime_denominators
    for i, idx in enumerate(stencil_idxs, start=1):
        stencil[idx] = np.sum(fractions[i:])

stencil

{-4: -3, -2: 27, 0: 47, 2: -13, 4: 2}

In [12]:
stencil_sum = sum(stencil[i] for i in stencil_idxs)
normalized_stencil = {i: stencil[i] / stencil_sum for i in stencil_idxs}
normalized_stencil

{-4: -0.05,
 -2: 0.45,
 0: 0.7833333333333333,
 2: -0.21666666666666667,
 4: 0.03333333333333333}

In [13]:
sum(normalized_stencil[i] for i in stencil_idxs)

1.0